# Running heat in parallel on a Jupyter Notebook

The original version of this tutorial was inspired by the [CS228 tutorial](https://github.com/kuleshov/cs228-material/blob/master/tutorials/python/cs228-python-tutorial.ipynb) by Volodomyr Kuleshov and Isaac Caswell.


<div style="float: center; padding-right: 2em; padding-top: 2em;">
    <img src="https://raw.githubusercontent.com/helmholtz-analytics/heat/master/doc/images/logo.png"></img>
</div>

## Installation

### Anaconda

This tutorial is designed to run on your local machine. Generally, to run Heat you need an [MPI](https://hpc-tutorials.llnl.gov/mpi/) installation, and a Python environment with PyTorch and mpi4py. The easiest way to set up such an environment is to install `heat` via conda: 

```shell
conda create --name heat-env python=3.11
conda activate heat-env
conda install -c conda-forge heat xarray jupyter scikit-learn
```

### Pip

#### OpenMPI

Heat can also be installed with pip, but ```openmpi``` has to be available on the system. To install ```openmpi``` on linux/macos:

```shell
# Ubuntu
$ sudo apt install openmpi-bin libopenmpi-dev

# Arch
$ sudo pacman -S openmpi

# MacOS
$ brew install openmpi
```

Or try other installing the binary. Instructions [here](https://docs.open-mpi.org/en/v5.0.x/installing-open-mpi/quickstart.html)

Once ```openmpi``` is installed, confirm that the installation worked by running

```console
$ ompi_info
```

#### Heat

Afterwards, you can create a virtual environment with pip, and install ```heat```.

```console
python -m venv heat-venv
source heat-venv/bin/activate
pip install heat[examples] xarray
```

## Setting up the IPyParallel environment

In this tutorial, we want to demonstrate how Heat distributes arrays and operations across multiple MPI processes. We can do this interactively in a Jupyter Notebook using IPyParallel.


Add your virtual environment to the Jupyter kernels:

```bash
python -m ipykernel install --user --name=heat-env
```

Start the Jupyter Notebook server:

```bash
jupyter notebook
```

Now you can select the `heat-env` kernel when creating a new notebook.

Finally, you need to start an IPyParallel cluster to access multiple MPI processes. You can do this by running the following command in a terminal:

```bash
ipcluster start -n 4 --engines=mpi
```

This will start an IPyParallel cluster with 4 MPI processes. You can adjust the number of processes as needed.

For a great intro on `ipyparallel` usage on our supercomputers, check out Jan Meinke's tutorial ["Interactive Parallel Computing with IPython Parallel"](https://gitlab.jsc.fz-juelich.de/sdlbio-courses/hpc-python/-/blob/master/06_LocalParallel.ipynb) or the [ipyparallel docs](https://ipyparallel.readthedocs.io/en/latest/).

Let's check the health of the available processes. 

In [ ]:
from ipyparallel import Client
rc = Client(profile="default")

Earlier, we have started an IPython cluster with 4 processes. We can now check if all 4 processes are available.

In [ ]:
rc.ids

[0, 1, 2, 3]

The `px` magic command allows you to execute Python commands or a Jupyter cell on the ipyparallel engines interactively ([%%px documentation](https://ipyparallel.readthedocs.io/en/latest/tutorial/magics.html)).

We can now finally import `heat` on our 4-process cluster.

In [ ]:
%px import heat as ht

%px:   0%|          | 0/4 [00:00<?, ?tasks/s]